In [128]:
import os, pickle
import numpy as np
from scipy.io import loadmat
from dataclasses import dataclass
from typing import List, Dict, Any
import h5py
import matplotlib.pyplot as plt

In [129]:
# path = "/home/dacap/Documents/p5control-bluefors-evaluation/theory/transmissions/MARS"
path = r"A:\Documents\Git\p5control-bluefors-evaluation\simulation\data\Reference"
files = os.listdir(path)
files = files[1:]
# print(files)

arr = loadmat(f"{path}/{files[10]}")

V = np.array(arr["eV_eff_range"][0], dtype="float64")
I_0 = np.full((len(files) + 1, len(V)), 0, dtype="float64")
I_m = np.full((len(files) + 1, 20, len(V)), 0, dtype="float64")
tau = np.full(len(files) + 1, 0, dtype="float64")
M = np.arange(1, 21)

# arr = loadmat(f'{path}/{files[80]}')


@dataclass
class DataSet:
    V0: np.ndarray
    I0: np.ndarray
    Im: List[np.ndarray]


sets: Dict[float, DataSet] = {}

for i, file in enumerate(files):
    arr = loadmat(f"{path}/{file}")

    sets[float(file[15:19])] = DataSet(
        V0=np.real(arr["eV_eff_range"][0]),
        I0=np.real(arr["I_In"][:, 0]) / 2,
        Im=[
            np.real(arr["In_final"][:, 20 - m] / 2 + arr["In_final"][:, 20 + m] / 2)
            for m in M
        ],
    )

In [130]:
def fill(arr, isX):
    index_neg=589
    if isX:
        start = np.copy(arr[:index_neg+1])
        end = np.copy(arr[index_neg+1:])
        mid = np.round(np.arange(-0.1, 0.11, 0.01),2)[1:-1]
        return np.concatenate((start, mid, end))
    else:
        start = np.copy(arr[:index_neg+1])
        end = np.copy(arr[index_neg+1:])
        mid = np.zeros(len(np.round(np.arange(-0.1, 0.11, 0.01),2)[1:-1]))
        return np.concatenate((start, mid, end))

In [131]:
# correct data gap [-0.1, 0.1]
for key, set in sets.items():
    sets[key] = DataSet(
        V0=fill(set.V0, True),
        I0=fill(set.I0, False),
        Im=[fill(im, False) for im in set.Im]
    )

In [132]:
# set = sets[0.5]
# for i in range(20):
#     plt.plot(set.V0, set.Im[i], label="I0")

In [133]:
# save sets as h5py file

with h5py.File("DavidOIV.hdf5", "w") as f:
    for key, dataset in sets.items():
        group = f.create_group(str(key))
        group.create_dataset("V0", data=dataset.V0)
        group.create_dataset("I0", data=dataset.I0)
        group.create_dataset(f"Im", data=dataset.Im)